In [7]:
from helpers import *
from simulator import *
from agents import *

In [8]:
x = ConnectN(7, 4)
x.print_grid()

[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]]


In [9]:
p1 = Q_Learner(x)
p2 = Random_Learner(x)

TypeError: __init__() takes at least 4 arguments (2 given)

In [ ]:
play_game_mod()